In [ ]:
import numpy as np
import itertools
import datetime

In [ ]:
params = {
    "lr": [0.001, 0.01, 0.1],
    "neg_adv": [True, False],
    "hidden_dim": [50, 100, 250, 400],
    "norm": ['l1', 'l2'],
    "regularization_coef": [2e-6, 2e-8],
    "gamma": [1, 5, 10, 20],
}

In [ ]:
def get_random_samples(params, n, replacement=False, seed=42):
    all_keys = params.keys()
    all_vals = [v for k,v in params.items()]
    combinations = list(itertools.product(*all_vals))
    
    rnd = np.random.RandomState(seed)
    res_list = [combinations[i] for i in rnd.choice(len(combinations), n, replace=replacement)]
    all_res = []
    
    for p in res_list:
        all_res.append( {k: p[i] for (i,k) in enumerate(all_keys)} )
        
    return all_res

In [ ]:
n = 22
samples = get_random_samples(params, n)

In [ ]:
# fixed params
DATA_PATH="./train"
SAVE_PATH="./data/results"
DATASET="COVID-19"
FORMAT="raw_udd_hrt"

LOG_INTERVAL=1000
BATCH_SIZE=2070
BATCH_SIZE_EVAL=1020
NEG_SAMPLE_SIZE=30

In [ ]:
import os

for idx, s in enumerate(samples):
    print(f"---TEST {idx + 1}---")
        
    model_name = f"TransE_{s['norm']}"
    neg_adv_flag = '-adv' if s['neg_adv'] else ''

    folder = '{}_{}_'.format(model_name, DATASET)
    n = len([x for x in os.listdir(SAVE_PATH) if x.startswith(folder)])
    folder += str(n)
    model_path = os.path.join(SAVE_PATH, folder)
    filename = f"./log/run_{idx + 1}_{folder}.txt"
    
    !DGLBACKEND=pytorch dglke_train --model_name {model_name} --data_path {DATA_PATH} --save_path {SAVE_PATH}  --dataset {DATASET}  --format {FORMAT} \
    --data_files train.tsv valid.tsv test.tsv \
    --log_interval {LOG_INTERVAL} --batch_size {BATCH_SIZE} --batch_size_eval {BATCH_SIZE_EVAL} --neg_sample_size {NEG_SAMPLE_SIZE} \
    --lr {s['lr']} {neg_adv_flag} --hidden_dim {s['hidden_dim']} -rc {s['regularization_coef']} -g {s['gamma']} \
    --gpu 0 --mix_cpu_gpu --max_step 10000
    
    !DGLBACKEND=pytorch dglke_eval --dataset {DATASET} --data_path {DATA_PATH} --data_files train.tsv valid.tsv test.tsv \
    --format {FORMAT} --model_name {model_name} --hidden_dim {s['hidden_dim']} \
    --gpu 0 --model_path {model_path} --batch_size_eval 1020 --neg_sample_size_eval 30 |& tee {filename}